In [ ]:
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import json
import ipywidgets as widgets
from IPython.display import display


# Sample data creation with more details
data = {
    "Project A": {
        "Day 1": [
            {"clicks": 10, "contribution": 5, "learning_curve": 3, "work_hours": 4, "user": "User1"},
            {"clicks": 12, "contribution": 6, "learning_curve": 4, "work_hours": 5, "user": "User2"}
        ],
        "Day 2": [
            {"clicks": 20, "contribution": 7, "learning_curve": 4, "work_hours": 5, "user": "User1"},
            {"clicks": 18, "contribution": 8, "learning_curve": 5, "work_hours": 6, "user": "User2"}
        ],
        "Day 3": [
            {"clicks": 15, "contribution": 6, "learning_curve": 5, "work_hours": 6, "user": "User1"},
            {"clicks": 22, "contribution": 9, "learning_curve": 6, "work_hours": 7, "user": "User2"}
        ],
        "Day 4": [
            {"clicks": 25, "contribution": 8, "learning_curve": 6, "work_hours": 7, "user": "User1"},
            {"clicks": 20, "contribution": 7, "learning_curve": 5, "work_hours": 6, "user": "User2"}
        ]
    },
    "Project B": {
        "Day 1": [
            {"clicks": 5, "contribution": 3, "learning_curve": 2, "work_hours": 3, "user": "User3"},
            {"clicks": 8, "contribution": 4, "learning_curve": 3, "work_hours": 4, "user": "User4"}
        ],
        "Day 2": [
            {"clicks": 15, "contribution": 5, "learning_curve": 4, "work_hours": 5, "user": "User3"},
            {"clicks": 12, "contribution": 6, "learning_curve": 5, "work_hours": 6, "user": "User4"}
        ],
        "Day 3": [
            {"clicks": 10, "contribution": 4, "learning_curve": 3, "work_hours": 4, "user": "User3"},
            {"clicks": 18, "contribution": 7, "learning_curve": 5, "work_hours": 5, "user": "User4"}
        ],
        "Day 4": [
            {"clicks": 30, "contribution": 9, "learning_curve": 5, "work_hours": 8, "user": "User3"},
            {"clicks": 25, "contribution": 8, "learning_curve": 6, "work_hours": 7, "user": "User4"}
        ]
    },
    "Project C": {
        "Day 1": [
            {"clicks": 7, "contribution": 4, "learning_curve": 3, "work_hours": 4, "user": "User5"},
            {"clicks": 10, "contribution": 5, "learning_curve": 4, "work_hours": 5, "user": "User6"}
        ],
        "Day 2": [
            {"clicks": 17, "contribution": 6, "learning_curve": 5, "work_hours": 5, "user": "User5"},
            {"clicks": 14, "contribution": 7, "learning_curve": 6, "work_hours": 6, "user": "User6"}
        ],
        "Day 3": [
            {"clicks": 12, "contribution": 5, "learning_curve": 4, "work_hours": 6, "user": "User5"},
            {"clicks": 18, "contribution": 8, "learning_curve": 5, "work_hours": 7, "user": "User6"}
        ],
        "Day 4": [
            {"clicks": 22, "contribution": 7, "learning_curve": 6, "work_hours": 7, "user": "User5"},
            {"clicks": 20, "contribution": 6, "learning_curve": 5, "work_hours": 6, "user": "User6"}
        ]
    }
}

"""
# Mount Google Drive
drive.mount('/content/drive')

# Save data as JSON file
with open('/content/drive/My Drive/project_data.json', 'w') as fid:
    json.dump(data, fid)

# Load data from JSON file
with open('/content/drive/My Drive/project_data.json', 'r') as fid:
    data = json.load(fid)
"""

# Convert data to pandas DataFrame
records = []
for project, days in data.items():
    for day, user_data in days.items():
        for metrics in user_data:
            record = {'Project': project, 'Day': day}
            record.update(metrics)
            records.append(record)

df = pd.DataFrame(records)
print(df)

# Dropdown for project
project_dropdown = widgets.Dropdown(
    options=df['Project'].unique().tolist(),
    description='Project:',
    disabled=False
)
display(project_dropdown)

# Dropdown for data type
data_type = widgets.Dropdown(
    options=['clicks', 'contribution', 'learning_curve', 'work_hours'],
    description='Data Type:',
    disabled=False
)
display(data_type)

# Dropdown for graph type
graph_type = widgets.Dropdown(
    options=['Bar', 'Line', 'Pie'],
    description='Graph Type:',
    disabled=False
)
display(graph_type)

# Output widget for displaying graph and summary
output = widgets.Output()
display(output)

# Function to update graph with selected graph type and data type
def update_graph(project, data_type, graph_type):
    with output:
        output.clear_output(wait=True)
        filtered_df = df[df['Project'] == project]

        if not filtered_df.empty:
            plt.figure(figsize=(10, 5))
            if graph_type == 'Bar':
                for user in filtered_df['user'].unique():
                    user_df = filtered_df[filtered_df['user'] == user]
                    plt.bar(user_df['Day'], user_df[data_type], label=user)
            elif graph_type == 'Line':
                for user in filtered_df['user'].unique():
                    user_df = filtered_df[filtered_df['user'] == user]
                    plt.plot(user_df['Day'], user_df[data_type], marker='o', linestyle='-', label=user)
            elif graph_type == 'Pie':
                user_totals = filtered_df.groupby('user')[data_type].sum()
                plt.pie(user_totals, labels=user_totals.index, autopct='%1.1f%%')
                plt.title(f'{data_type.capitalize().replace("_", " ")} Distribution for {project}')

            plt.xlabel('Day')
            plt.ylabel(data_type.capitalize().replace('_', ' '))
            plt.title(f'{data_type.capitalize().replace("_", " ")} for {project}')
            if graph_type != 'Pie':
                plt.legend(title='User')
                plt.grid(axis='y')
            plt.show()
        else:
            print(f"No data available for {project}")

# Function to display summary of the data
def display_summary(project, data_type):
    with output:
        output.clear_output(wait=True)
        filtered_df = df[df['Project'] == project]

        if not filtered_df.empty:
            display(filtered_df[['Day', 'Project', 'user', data_type]])
        else:
            print(f"No data available for {project}")

# Function to handle button click event
def on_button_click(b):
    display_summary(project_dropdown.value, data_type.value)
    update_graph(project_dropdown.value, data_type.value, graph_type.value)

# Button to show graph and data summary
button = widgets.Button(description="Show Graph")
display(button)
button.on_click(on_button_click)


      Project    Day  clicks  contribution  learning_curve  work_hours   user
0   Project A  Day 1      10             5               3           4  User1
1   Project A  Day 1      12             6               4           5  User2
2   Project A  Day 2      20             7               4           5  User1
3   Project A  Day 2      18             8               5           6  User2
4   Project A  Day 3      15             6               5           6  User1
5   Project A  Day 3      22             9               6           7  User2
6   Project A  Day 4      25             8               6           7  User1
7   Project A  Day 4      20             7               5           6  User2
8   Project B  Day 1       5             3               2           3  User3
9   Project B  Day 1       8             4               3           4  User4
10  Project B  Day 2      15             5               4           5  User3
11  Project B  Day 2      12             6               5      

Dropdown(description='Project:', options=('Project A', 'Project B', 'Project C'), value='Project A')

Dropdown(description='Data Type:', options=('clicks', 'contribution', 'learning_curve', 'work_hours'), value='…

Dropdown(description='Graph Type:', options=('Bar', 'Line', 'Pie'), value='Bar')

Output()

Button(description='Show Graph', style=ButtonStyle())